# Prediction Potensial Customer

## Import Libraries

In [1]:
!pip install tfx tensorflow_model_analysis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import pandas as pd
import tensorflow_model_analysis as tfma
from tfx.types import Channel
from tfx.dsl.components.common.resolver import Resolver
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.components import Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy

## Dataset

In [4]:
!unzip '/content/data.zip'

Archive:  /content/data.zip
replace data/bank-marketing.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [5]:
df = pd.read_csv('/content/data/bank-marketing.csv')
df.head()

,age,age group,eligible,job,salary,marital,education,marital-education,targeted,default,...,contact,day,month,duration,campaign,pdays,previous,poutcome,y,response
0,58,5,Y,management,100000,married,tertiary,married-tertiary,yes,no,...,unknown,5,may,261,1,-1,0,unknown,no,0
1,44,4,Y,technician,60000,single,secondary,single-secondary,yes,no,...,unknown,5,may,151,1,-1,0,unknown,no,0
2,33,3,Y,entrepreneur,120000,married,secondary,married-secondary,yes,no,...,unknown,5,may,76,1,-1,0,unknown,no,0
3,47,4,Y,blue-collar,20000,married,unknown,married-unknown,no,no,...,unknown,5,may,92,1,-1,0,unknown,no,0
4,33,3,Y,unknown,0,single,unknown,single-unknown,no,no,...,unknown,5,may,198,1,-1,0,unknown,no,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   age                45211 non-null  int64 
 1   age group          45211 non-null  int64 
 2   eligible           45211 non-null  object
 3   job                45211 non-null  object
 4   salary             45211 non-null  int64 
 5   marital            45211 non-null  object
 6   education          45211 non-null  object
 7   marital-education  45211 non-null  object
 8   targeted           45211 non-null  object
 9   default            45211 non-null  object
 10  balance            45211 non-null  int64 
 11  housing            45211 non-null  object
 12  loan               45211 non-null  object
 13  contact            45211 non-null  object
 14  day                45211 non-null  int64 
 15  month              45211 non-null  object
 16  duration           45211 non-null  int64

## Components.py

In [7]:
# Mendefinisikan nama module 
COMPONENTS_MODULE_FILE = 'modules/components.py'

In [8]:
%%writefile {COMPONENTS_MODULE_FILE}

import os

import tensorflow_model_analysis as tfma
from tfx.components import (CsvExampleGen, Evaluator, ExampleValidator, Pusher,
                            SchemaGen, StatisticsGen, Trainer, Transform,
                            Tuner)
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import \
    LatestBlessedModelStrategy
from tfx.proto import example_gen_pb2, pusher_pb2, trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing


def init_components(args):
    """Initiate tfx pipeline components

    Args:
        args (dict): args that containts some dependencies

    Returns:
        tuple: TFX pipeline components
    """
    output = example_gen_pb2.Output(
        split_config=example_gen_pb2.SplitConfig(splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
        ])
    )

    example_gen = CsvExampleGen(
        input_base=args["data_dir"],
        output_config=output,
    )

    statistics_gen = StatisticsGen(
        examples=example_gen.outputs["examples"],
    )

    schema_gen = SchemaGen(
        statistics=statistics_gen.outputs["statistics"],
    )

    example_validator = ExampleValidator(
        statistics=statistics_gen.outputs["statistics"],
        schema=schema_gen.outputs["schema"],
    )

    transform = Transform(
        examples=example_gen.outputs["examples"],
        schema=schema_gen.outputs["schema"],
        module_file=os.path.abspath(args["transform_module"]),
    )

    tuner = Tuner(
        module_file=os.path.abspath(args["tuner_module"]),
        examples=transform.outputs["transformed_examples"],
        transform_graph=transform.outputs["transform_graph"],
        schema=schema_gen.outputs["schema"],
        train_args=trainer_pb2.TrainArgs(
            splits=["train"],
            num_steps=args["train_steps"],
        ),
        eval_args=trainer_pb2.EvalArgs(
            splits=["eval"],
            num_steps=args["eval_steps"],
        ),
    )

    trainer = Trainer(
        module_file=args["trainer_module"],
        examples=transform.outputs["transformed_examples"],
        transform_graph=transform.outputs["transform_graph"],
        schema=schema_gen.outputs["schema"],
        hyperparameters=tuner.outputs["best_hyperparameters"],
        train_args=trainer_pb2.TrainArgs(
            splits=["train"],
            num_steps=args["train_steps"],
        ),
        eval_args=trainer_pb2.EvalArgs(
            splits=["eval"],
            num_steps=args["eval_steps"]
        ),
    )

    model_resolver = Resolver(
        strategy_class=LatestBlessedModelStrategy,
        model=Channel(type=Model),
        model_blessing=Channel(type=ModelBlessing),
    ).with_id("Latest_blessed_model_resolve")

    eval_config = tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key="response")],
        slicing_specs=[
            tfma.SlicingSpec()#,
            #tfma.SlicingSpec(feature_keys=["targeted"]),
        ],
        metrics_specs=[
            tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name="AUC"),
                tfma.MetricConfig(class_name="Precision"),
                tfma.MetricConfig(class_name="Recall"),
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name="TruePositives"),
                tfma.MetricConfig(class_name="FalsePositives"),
                tfma.MetricConfig(class_name="TrueNegatives"),
                tfma.MetricConfig(class_name="FalseNegatives"),
                tfma.MetricConfig(
                    class_name="BinaryAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": .6},
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={"value": 1e-4},
                        ),
                    ),
                ),
            ]),
        ],
    )

    evaluator = Evaluator(
        examples=example_gen.outputs["examples"],
        model=trainer.outputs["model"],
        baseline_model=model_resolver.outputs["model"],
        eval_config=eval_config,
    )

    pusher = Pusher(
        model=trainer.outputs["model"],
        model_blessing=evaluator.outputs["blessing"],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory=args["serving_model_dir"],
            ),
        ),
    )

    return (
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        tuner,
        trainer,
        model_resolver,
        evaluator,
        pusher,
    )


Overwriting modules/components.py


## Transform Modules

In [9]:
# Mendefinisikan nama module 
TRANSFORM_MODULE_FILE = 'modules/transform.py'

In [10]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft

CATEGORICAL_FEATURES = {
    'eligible' : 2,
    'job' : 12,
    'marital' : 3,
    'education'	: 4,
    'targeted' : 2,
    'default' : 2,
    'housing' : 2,
    'loan' : 2,
    'month' : 12
}

NUMERICAL_FEATURES = [
    'age', 
    'salary', 
    'balance', 
    'duration', 
    'day', 
    'campaign', 
    'pdays', 
    'previous'    
]

LABEL_KEY = 'response'


def transformed_name(key):
    """Transform feature key

    Args:
        key (str): the key to be transformed

    Returns:
        str: transformed key
    """

    return f"{key}_xf"


def convert_num_to_one_hot(label_tensor, num_labels=2):
    """Convert a label (0 or 1) into a one-hot vector

    Args:
        label_tensor (int): label tensor (0 or 1)
        num_labels (int, optional): num of label. Defaults to 2.

    Returns:
        tf.Tensor: label tensor
    """

    one_hot_tensor = tf.one_hot(label_tensor, num_labels)
    return tf.reshape(one_hot_tensor, [-1, num_labels])



def preprocessing_fn(inputs):
    """Preprocess input features into transformed features

    Args:
        inputs (dict): map from feature keys to raw features

    Returns:
        dict: map from features keys to transformed features
    """

    outputs = {}

    for keys, values in CATEGORICAL_FEATURES.items():
        int_value = tft.compute_and_apply_vocabulary(
            inputs[keys], top_k=values+1)
        outputs[transformed_name(keys)] = convert_num_to_one_hot(
            int_value, num_labels=values+1)

    for feature in NUMERICAL_FEATURES:
        outputs[transformed_name(feature)] = tft.scale_to_0_1(inputs[feature])

    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs


Overwriting modules/transform.py


## Tuner Modules

In [11]:
# Mendefinisikan nama module 
TUNER_MODULE_FILE = 'modules/tuner.py'

In [12]:
%%writefile {TUNER_MODULE_FILE}

from typing import Any, Dict, NamedTuple, Text

import keras_tuner as kt
import tensorflow as tf
import tensorflow_transform as tft
from keras import layers
from keras_tuner.engine import base_tuner
from transform import (CATEGORICAL_FEATURES, LABEL_KEY, NUMERICAL_FEATURES,
                       transformed_name)

NUM_EPOCHS = 5

TunerFnResult = NamedTuple("TunerFnResult", [
    ("tuner", base_tuner.BaseTuner),
    ("fit_kwargs", Dict[Text, Any]),
])

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_binary_accuracy",
    mode="max",
    verbose=1,
    patience=10,
)


def gzip_reader_fn(filenames):
    """Loads compression data

    Args:
        filenames (str): a path to the data directory

    Returns:
        TfRecord: Compressed data
    """

    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_fn(file_pattern, tf_transform_output, batch_size=64):
    """Generated features and labels for tuning/training

    Args:
        file_pattern: input tf_record file pattern
        tf_transform_output: a TFTransformOutput
        batch_size: representing the number of consecutive elements of
        returned dataset to combine in a single batch. Defaults to 64.

    Returns:
        a dataset that contains (featurs, indices) tuple where features
        is a dictionary of Tensors, and indices is a single Tensor of
        label indices
    """

    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        label_key=transformed_name(LABEL_KEY)
    )

    return dataset


def get_model_tuner(hyperparameters):
    """This function defines a keras Model

    Args:
        hyperparameters (kt.HyperParameters): object to setting hyperparameters

    Returns:
        tf.keras.Model: model as a Keras object
    """

    num_hidden_layers = hyperparameters.Choice(
        "num_hidden_layers",
        values=[1, 2, 3],
    )
    dense_unit = hyperparameters.Int(
        "dense_unit",
        min_value=16,
        max_value=512,
        step=16,
    )
    dropout_rate = hyperparameters.Float(
        "dropout_rate",
        min_value=0.1,
        max_value=0.8,
        step=0.05,
    )
    learning_rate = hyperparameters.Choice(
        "learning_rate",
        values=[1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5, 5e-6, 1.e-6]
    )

    input_features = []

    for key, dim in CATEGORICAL_FEATURES.items():
        input_features.append(
            layers.Input(shape=(dim+1,), name=transformed_name(key))
        )

    for feature in NUMERICAL_FEATURES:
        input_features.append(
            layers.Input(shape=(1,), name=transformed_name(feature))
        )

    concatenate = layers.concatenate(input_features)
    deep = layers.Dense(dense_unit, activation=tf.nn.relu)(concatenate)

    for _ in range(num_hidden_layers):
        deep = layers.Dense(dense_unit, activation=tf.nn.relu)(deep)
        deep = layers.Dropout(dropout_rate)(deep)

    outputs = layers.Dense(1, activation=tf.nn.sigmoid)(deep)

    model = tf.keras.Model(inputs=input_features, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=["binary_accuracy"],
    )

    model.summary()

    return model


def tuner_fn(fn_args):
    """Tuning the model to get the best hyperparameters based on given args

    Args:
        fn_args (FnArgs): Holds args used to train the model as name/value pair

    Returns:
        TunerFnResult (NamedTuple): object to run model tuner
    """

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_set = input_fn(fn_args.train_files[0], tf_transform_output,)
    eval_set = input_fn(fn_args.eval_files[0], tf_transform_output,)

    tuner = kt.Hyperband(
        hypermodel=get_model_tuner,
        objective=kt.Objective(
            "val_loss",
            direction="min",
        ),
        max_epochs=NUM_EPOCHS,
        factor=3,
        directory=fn_args.working_dir,
        project_name="kt_hyperband",
    )

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "x": train_set,
            "validation_data": eval_set,
            "steps_per_epoch": fn_args.train_steps,
            "validation_steps": fn_args.eval_steps,
            "callbacks": [early_stop]
        },
    )

Overwriting modules/tuner.py


## Trainer Modules

In [13]:
# Mendefinisikan nama module 
TRAINER_MODULE_FILE = 'modules/trainer.py'

In [14]:
%%writefile {TRAINER_MODULE_FILE}
import os

import tensorflow as tf
import tensorflow_transform as tft
from keras import layers
from transform import (CATEGORICAL_FEATURES, LABEL_KEY, NUMERICAL_FEATURES,
                       transformed_name)
from tuner import input_fn


def get_serve_tf_examples_fn(model, tf_transform_output):
    """Return a function that parses a serialized tf.Example"""

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None], dtype=tf.string, name="examples"),
    ])
    def serve_tf_examples_fn(serialized_tf_examples):
        """Return the output to be used in the serving signature."""

        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(
            serialized_tf_examples, feature_spec,
        )

        transformed_features = model.tft_layer(parsed_features)
        outputs = model(transformed_features)

        return {"outputs": outputs}

    return serve_tf_examples_fn


def get_model(hyperparameters):
    """This model defines a keras Model

    Args:
        hyperparameters (kt.HyperParameters): object that contains best hyperparameters
        from tuner

    Returns:
        tf.keras.Model: model as a Keras object
    """

    input_features = []

    for key, dim in CATEGORICAL_FEATURES.items():
        input_features.append(
            layers.Input(shape=(dim+1,), name=transformed_name(key))
        )

    for feature in NUMERICAL_FEATURES:
        input_features.append(
            layers.Input(shape=(1,), name=transformed_name(feature))
        )

    concatenate = layers.concatenate(input_features)
    deep = layers.Dense(
        hyperparameters["dense_unit"], activation=tf.nn.relu)(concatenate)

    for _ in range(hyperparameters["num_hidden_layers"]):
        deep = layers.Dense(
            hyperparameters["dense_unit"], activation=tf.nn.relu)(deep)
        deep = layers.Dropout(hyperparameters["dropout_rate"])(deep)

    outputs = layers.Dense(1, activation=tf.nn.sigmoid)(deep)

    model = tf.keras.Model(inputs=input_features, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hyperparameters["learning_rate"]),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=["binary_accuracy"],
    )

    model.summary()

    return model


def run_fn(fn_args):
    """Train the model based on given args

    Args:
        fn_args (FnArgs): Holds args used to train the model as name/value pairs.
    """

    hyperparameters = fn_args.hyperparameters["values"]
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), "logs")

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_set = input_fn(fn_args.train_files, tf_transform_output)
    eval_set = input_fn(fn_args.eval_files, tf_transform_output)

    model = get_model(hyperparameters)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        update_freq="batch"
    )

    early_stop_callbacks = tf.keras.callbacks.EarlyStopping(
        monitor="val_binary_accuracy",
        mode="max",
        verbose=1,
        patience=10,
    )

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor="val_binary_accuracy",
        mode="max",
        verbose=1,
        save_best_only=True,
    )

    callbacks = [
        tensorboard_callback,
        early_stop_callbacks,
        model_checkpoint_callback
    ]

    model.fit(
        x=train_set,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_set,
        validation_steps=fn_args.eval_steps,
        callbacks=callbacks,
        epochs=hyperparameters["tuner/initial_epoch"],
        verbose=1,
    )

    signatures = {
        "serving_default": get_serve_tf_examples_fn(
            model, tf_transform_output,
        )
    }

    model.save(
        fn_args.serving_model_dir,
        save_format="tf",
        signatures=signatures,
    )

Overwriting modules/trainer.py


## Pipeline.py

In [15]:
# Mendefinisikan nama module 
PIPELINES_MODULE_FILE = 'modules/pipeline.py'

In [16]:
%%writefile {PIPELINES_MODULE_FILE}

from typing import Text

from absl import logging
from tfx.orchestration import metadata, pipeline


def init_pipeline(pipeline_root: Text, pipeline_name, metadata_path, components):
    """Initiate tfx pipeline

    Args:
        pipeline_root (Text): a path to th pipeline directory
        pipeline_name (str): pipeline name
        metadata_path (str): a path to the metadata directory
        components (dict): tfx components

    Returns:
        pipeline.Pipeline: pipeline orchestration
    """

    logging.info(f"Pipeline root set to: {pipeline_root}")

    beam_args = [
        "--direct_running_mode=multi_processing",
        "----direct_num_workers=0",
    ]

    return pipeline.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path,
        ),
        eam_pipeline_args=beam_args,
    )

Overwriting modules/pipeline.py


## Run Pipeline

In [17]:
#! pip install tfx tensorflow_model_analysis

In [18]:
import os
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from modules import components, pipeline

### Set Variabel


In [19]:
PIPELANE_NAME = "purchase-predict-pipeline"

# Pipeline inputs
DATA_ROOT = "/content/data"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TUNER_MODULE_FILE = "modules/tuner.py"
TRAINER_MODULE_FILE = "modules/trainer.py"

# Pipeline outputs
OUTPUT_BASE = "outputs"

serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELANE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

### Menjalankan ML Pipeline

In [20]:
components_args = {
    "data_dir": DATA_ROOT,
    "trainer_module": TRAINER_MODULE_FILE,
    "tuner_module": TUNER_MODULE_FILE,
    "transform_module": TRANSFORM_MODULE_FILE,
    "train_steps": 1000,
    "eval_steps": 800,
    "serving_model_dir": serving_model_dir,
}

components = components.init_components(components_args)

pipeline = pipeline.init_pipeline(
    pipeline_root, 
    PIPELANE_NAME, 
    metadata_path, 
    components
)
BeamDagRunner().run(pipeline)

Trial 10 Complete [00h 01m 23s]
val_loss: 0.23939529061317444

Best val_loss So Far: 0.23788103461265564
Total elapsed time: 00h 06m 12s
Results summary
Results in outputs/purchase-predict-pipeline/Tuner/.system/executor_execution/21/.temp/21/kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
num_hidden_layers: 1
dense_unit: 32
dropout_rate: 0.40000000000000013
learning_rate: 0.001
tuner/epochs: 5
tuner/initial_epoch: 2
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0000
Score: 0.23788103461265564
Trial summary
Hyperparameters:
num_hidden_layers: 3
dense_unit: 512
dropout_rate: 0.5500000000000002
learning_rate: 0.001
tuner/epochs: 5
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.23939529061317444
Trial summary
Hyperparameters:
num_hidden_layers: 3
dense_unit: 64
dropout_rate: 0.15000000000000002
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.24364684522151947
Trial summary
Hyperparameters:
num_hidde

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 eligible_xf (InputLayer)       [(None, 3)]          0           []                               
                                                                                                  
 job_xf (InputLayer)            [(None, 13)]         0           []                               
                                                                                                  
 marital_xf (InputLayer)        [(None, 4)]          0           []                               
                                                                                                  
 education_xf (InputLayer)      [(None, 5)]          0           []                               
                                                                                            

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
